#### Data Loading and Transformation

The provided code snippet demonstrates how we load image data using PyTorch's `ImageFolder` and `DataLoader` modules, while also applying custom preprocessing transformations.

1. **Custom Transformations:**
   - Two custom transformation classes are defined:
     - **Denoise:** Applies Gaussian blur to the image using `ImageFilter.GaussianBlur`.
     - **HistogramEqualization:** Performs histogram equalization on the image. If the image is RGB, it converts it to grayscale (`cv2.COLOR_RGB2GRAY`) before equalizing the histogram.

2. **Transformation Pipeline:**
   - A `transforms.Compose` object chains together several transformations:
     - `HistogramEqualization()`: Custom transformation to enhance image contrast.
     - `Denoise()`: Custom transformation to reduce noise using Gaussian blur.
     - `transforms.Resize((224, 224))`: Resizes the image to a fixed size of 224x224 pixels.
     - `transforms.ToTensor()`: Converts the PIL Image to a PyTorch tensor.
     - (Commented out) `transforms.Normalize()`: Normalization step which is currently disabled.

3. **Dataset Loading:**
   - `ImageFolder` is used to create a dataset from images located at the specified `path`, applying the defined transformations.
   - `DataLoader` is then used to create batches of the dataset for training or evaluation.

4. **Subset (Optional):**
   - The code includes a commented-out line (`Subset`) which can be used to create a subset of the dataset if needed.

5. **Batch Size and Subset Size:**
   - Parameters `batch_size` and `subset_size` control the size of batches and the subset of the dataset to load, respectively.

This setup prepares the image data for further processing or training using PyTorch, with the flexibility to add or modify transformations as needed.

We utilized this code to customize our images and aimed to achieve optimal results. 
To achieve this, we experimented with the `transforms.Compose` function by adding and removing transformations within it.

In [2]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import functional as F
import numpy as np
import cv2
from PIL import ImageFilter

class Denoise:
    def __call__(self, img):
        img = img.filter(ImageFilter.GaussianBlur(radius=1))
        return img

class HistogramEqualization:
    def __call__(self, img):
        img = np.array(img)
        if len(img.shape) == 3 and img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.equalizeHist(img)
        return F.to_pil_image(img)


def load_data(path, subset_size=624, batch_size=64):
    transform = transforms.Compose([
        HistogramEqualization(),
        Denoise(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485], std=[0.229])
    ])

    dataset_folder = ImageFolder(root=path, transform=transform)
    # subset = Subset(dataset_folder, range(subset_size))
    dataset = DataLoader(dataset_folder, batch_size=batch_size)

    # print('Dataset\t', 'Train\t', dataset_folder.classes[0], '', dataset_folder.classes[1])
    # print('Total:\t', len(dataset_folder), '\t', dataset_folder.targets.count(0), '\t', dataset_folder.targets.count(1))

    return dataset

#### Our method: Visualizing and Improving Data Transformations

We utilized a methodical approach to enhance image data transformations:

1. **Visualization and Adjustment:**  
   We converted images to PNG format to visually assess the effects of transformations like denoising and histogram equalization.

2. **Human-Centric Evaluation:**  
   This method allowed for straightforward evaluation of transformation impacts, guiding iterative adjustments to our `transforms.Compose` pipeline.

3. **Iterative Refinement:**  
   By iteratively refining transformations based on visual feedback, we optimized our image preprocessing for better performance in machine learning tasks.

In [ ]:
import os
from torchvision import transforms
from tqdm import tqdm

def save_transformed_images(dataset, path):
    os.makedirs(path, exist_ok=True)
    for i, (image, label) in enumerate(tqdm(dataset)):
        image = transforms.ToPILImage()(image)
        image.save(os.path.join(path, f'imagey_{i}_label_{label}.png'))

data_path = 'datasets/train'

dataset = load_data(path=data_path, batch_size=64)

save_transformed_images(dataset.dataset, 'datasets/train_transformed/NORMAL')

Example of to images before and after applying transformations:

### **Original Image:**
  ![Original Image](resources/jupiter/normal.png)

### **Transformed Image:**
  ![Transformed Image](resources/jupiter/transformed.png)

By following this method, we were able to fine-tune our data transformations effectively.

#### Network

Here is the code of the network we used for training

In [ ]:
from torch import nn

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 28 * 28)  # Flatten the output of conv3 layer
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


### Training Method Summary

1. **Training Loop:**
   - Trains a CNN model on the provided dataset for a specified number of epochs.
   - Uses CrossEntropyLoss and Adam optimizer.
   - Tracks and prints training loss and accuracy.

2. **Model Saving:**
   - Saves the trained model state to a file specified by the user or defaults to 'model.pth'.

3. **Device Setup:**
   - Utilizes GPU if available, otherwise falls back to CPU.

4. **Execution:**
   - Loads the dataset.
   - Initializes the model.
   - Trains the model.
   - Saves the trained model.

In [ ]:
import torch
from tqdm import tqdm
import sys

def train_model(dataset, model, num_epochs=20):
    # Define the loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.train()
    accuracy_total_train = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        progress_bar = tqdm(total=len(dataset), desc='Epoch {}/{}'.format(epoch+1, num_epochs), position=0, leave=True)

        for inputs, labels in dataset:
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update the weights
            running_loss += loss.item() * inputs.size(0)

            outputs = torch.nn.functional.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            accuracy_total_train.append(torch.sum(preds == labels.data).item() / float(inputs.size(0)))

            progress_bar.update(1)

        epoch_loss = running_loss / len(dataset)
        progress_bar.close()
        print('Loss: {:.4f}'.format(epoch_loss), 'Accuracy: {:.4f}'.format(sum(accuracy_total_train) / len(accuracy_total_train)))

    print('Finished Training')
    return model

model_name = 'model.pth'
if len(sys.argv) > 1:
    model_name = sys.argv[1]

# Load datasets
dataset = load_data('datasets/train_transformed', subset_size=5216, batch_size=128)

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device, '\n')

# Create an instance of the model
num_classes = len(dataset.dataset.classes)
model = CNN(num_classes=num_classes).to(device)

# Training loop
model = train_model(dataset, model, num_epochs=5)

print('Saving the model...')
torch.save(model.state_dict(), model_name)
print('Model saved as', model_name)

### Testing Method Summary

1. **Testing Loop:**
   - Evaluates the trained model on the test dataset.
   - Computes and prints the test accuracy.

2. **Device Setup:**
    - Utilizes GPU if available, otherwise falls back to CPU.

3. **Execution:**
    - Loads the test dataset.
    - Initializes the model.
    - Loads the weights of the trained model.
    - Tests the model on the test dataset.

In [ ]:
import torch
from tqdm import tqdm
import sys

model_name = 'model.pth'
if len(sys.argv) > 1:
    model_name = sys.argv[1]

dataset = load_data(path='datasets/test', batch_size=64)

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device, '\n')

# Create an instance of the model
num_classes = len(dataset.dataset.classes)
model = CNN(num_classes=num_classes).to(device)

print('Loading the model', model_name + '...')
model.load_state_dict(torch.load(model_name))
model.eval()

# Test the model
correct = 0
total = 0

progress_bar = tqdm(total=len(dataset), desc='Testing', position=0, leave=True)

with torch.no_grad():
    for inputs, labels in dataset:
        logits = model.forward(inputs)
        _, predicted = torch.max(logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        progress_bar.update(1)

progress_bar.close()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))


### Model Evaluation

The model evaluation process is based on testing the trained model on unseen data to make sure it generalizes well. The test accuracy provides insights into the model's performance on new data.

### Conclusion

To conclude, the use of a CNN and proper data transformations can significantly improve the performance of patern recognition. To improve the model's performance, we experimented with various data transformations, but finetuning the network architecture could be investigated.